### **Препроцессинг фотковых текстов**

#### **1. Начальная обработка**

In [ ]:
import re

input_file = "recognized_all_gpt.txt"
output_file = "recognized_all_gpt_new.txt"

with open(input_file, "r", encoding="utf-8") as f:
    text = f.read()

text = re.sub(r'-\s*\n\s*', '', text)
text = re.sub(r'\n+', ' ', text)
text = re.sub(r'\s+', ' ', text).strip()
sentences = re.split(r'(?<=[.!?])(?:\s+|(?=["“”\'»)])|(?=\Z))', text)
sentences = [s.strip() for s in sentences if s.strip()]

with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(sentences))

print(f"Количество предложений: {len(sentences)}")

In [ ]:
import re

def clean_text(text):
    """
    Очищает текст от всех знаков препинания, кроме точки, и удаляет цифры.
    Каждое исходное предложение остается на новой строке.
    """
    lines = text.splitlines()  # Разделяем текст на строки (предложения)
    cleaned_lines = []

    for line in lines:

        line = re.sub(r"[!\"#$%&'()*+,-\/:;<=>?@\[\\\]^_`{|}~–«»…]+", "", line)
        line = re.sub(r"\d+", "", line)

        # удаляем лишние пробелы
        line = re.sub(r" +", " ", line)
        line = line.strip()


        if line:
            if not line.endswith('.'):
                 if line:
                      line += '.'
            cleaned_lines.append(line)


    return '\n'.join(cleaned_lines)

# Имена файлов
input_filename = 'recognized_all_gpt_new.txt'
output_filename = 'not_punct.txt'

try:

    with open(input_filename, 'r', encoding='utf-8') as infile:
        original_text = infile.read()


    cleaned_text = clean_text(original_text)

    with open(output_filename, 'w', encoding='utf-8') as outfile:
        outfile.write(cleaned_text)

    print(f"Файл '{input_filename}' успешно обработан.")
    print(f"Результат сохранен в файл '{output_filename}'.")

except FileNotFoundError:
    print(f"Ошибка: Файл '{input_filename}' не найден.")
    print("Пожалуйста, убедитесь, что файл находится в той же папке, что и скрипт, или укажите правильный путь.")

except Exception as e:
    print(f"Произошла ошибка при обработке файла: {e}")


In [ ]:
import re

def clean_text_revised(text):
    """
    Очищение текста:
    1. Удаление последовательности 'd''' и 'l'''.
    2. Только буквы (из любого алфавита) и пробелы.
    3. Нет цифр и пунктуации.
    4. Нормальные пробелы
    Каждая исходная строка остается на новой строке в результате.
    """
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        processed_line = line.replace("d'", "")
        processed_line = processed_line.replace("l'", "")

        # Оставить толкьо буквы и пробелы
        filtered_chars = [char for char in processed_line if char.isalpha() or char.isspace()]
        filtered_line = "".join(filtered_chars)
        # всё с одним пробелом 
        final_line = re.sub(r'\s+', ' ', filtered_line)
        final_line = final_line.strip()

        # Добавляем непустые строки
        if final_line:
            cleaned_lines.append(final_line)

    return '\n'.join(cleaned_lines)

input_filename = 'recognized_all_gpt_new.txt'
output_filename = 'not_punct.txt'

try:
    with open(input_filename, 'r', encoding='utf-8') as infile:
        original_text = infile.read()

    cleaned_text = clean_text_revised(original_text)

    with open(output_filename, 'w', encoding='utf-8') as outfile:
        outfile.write(cleaned_text)

    print(f"Файл '{input_filename}' успешно обработан.")
    print(f"Текст после удаления 'd\'', 'l\'', цифр и пунктуации сохранен в файл '{output_filename}'.")

except FileNotFoundError:
    print(f"Ошибка: Файл '{input_filename}' не найден.")
    print("Что-то не так с путём")
except Exception as e:
    print(f"Произошла ошибка при обработке файла: {e}")

#### **2. Сохранение именованных сущностей (топонимы) в отдельный файл**

In [ ]:
import re

def find_entities(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    entities = set()
    sentences = text.split('\n')
    
    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence)
        for i, word in enumerate(words):
            #считаем именованной сущностью слово, если оно начинается с заглавной латинской буквы
            if len(word) > 1 and word[0].isupper() and word.isascii():
                # фильтр по длине
                if len(word) > 2:
                    entities.add(word)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sorted(entities)))

if __name__ == "__main__":
    input_file = 'not_punct.txt'          
    output_file = 'named_entities.txt'

    find_entities(input_file, output_file)
    print(f'Результаты в {output_file}')

#### **3. Лемматизация файла с помощью парсера urmi-parser**

In [ ]:
from uniparser_urmi import UrmiAnalyzer

def tokenize_and_lemmatize(input_file, output_file):
    """
    Токенизирует и лемматизирует предложения из файла.
    Если лемма не найдена для слова, начинающегося на "в", удаляет "в" и пробует ещё раз.
    """
    a = UrmiAnalyzer(mode='nodiacritics')
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        sentences = [line.strip() for line in infile if line.strip()]
    
    results = []
    for sentence in sentences:
        lemmatized_sentence = []
        for word in sentence.split():
            analyses = a.analyze_words(word)
            lemmatized_word_analyses = []
            lemma_found = False
            
            for ana in analyses:
                if hasattr(ana, 'lemma') and ana.lemma:
                    lemmatized_word_analyses.append({
                        "token": ana.wf if hasattr(ana, 'wf') else word,
                        "lemma": ana.lemma,
                        "gramm": ana.gramm if hasattr(ana, 'gramm') else "",
                    })
                    lemma_found = True
                    break
            
            if not lemma_found and word.lower().startswith('в') and len(word) > 1:
                modified_word = word[1:]
                analyses_mod = a.analyze_words(modified_word)
                for ana in analyses_mod:
                    if hasattr(ana, 'lemma') and ana.lemma:
                        lemmatized_word_analyses.append({
                            "token": ana.wf if hasattr(ana, 'wf') else modified_word,
                            "lemma": ana.lemma,
                            "gramm": ana.gramm if hasattr(ana, 'gramm') else "",
                        })
                        lemma_found = True
                        break
            
            if not lemma_found:
                lemmatized_word_analyses.append({
                    "token": word,
                    "lemma": "",
                    "gramm": "", 
                })
            
            lemmatized_sentence.append({
                "word": word,
                "analyses": lemmatized_word_analyses,
            })
        
        results.append({
            "sentence": sentence,
            "lemmatized_words": lemmatized_sentence,
        })
    
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for item in results:
            outfile.write(f"Предложение: {item['sentence']}\n")
            for word_info in item["lemmatized_words"]:
                outfile.write(f"  Слово: {word_info['word']}\n")
                for analysis in word_info["analyses"]:
                    outfile.write(f"    Токен: {analysis.get('token', '')}\n") 
                    outfile.write(f"      Лемма: {analysis.get('lemma', '')}\n")
                    outfile.write(f"      Грамматика: {analysis.get('gramm', '')}\n")
            outfile.write("\n")

if __name__ == "__main__":
    input_file = 'not_punct.txt'
    output_file = 'lemmatized_not_punct_all.txt'
    
    print(f"Запуск токенизации и лемматизации для файла '{input_file}'...")
    tokenize_and_lemmatize(input_file, output_file)
    print(f"Обработка завершена. Результаты в '{output_file}'")

запись нелемматизированных слов в отдельный файл

In [ ]:
def find_words_without_lemma(input_file, output_file):
    """
    Файл с not_lemma
    Находит и записывает в файл слова без леммы (не короче 4 символов) из файла output_lemmatized_all.txt.
    """

    words_without_lemma = []
    current_word = None
    lemma_found = False

    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()

            if line.startswith("Слово:"):
                if current_word and not lemma_found and len(current_word) >= 4:
                    words_without_lemma.append(current_word)
                current_word = line.split(": ", 1)[1]
                lemma_found = False
            elif line.startswith("Лемма:") and current_word:
                parts = line.split(": ", 1)
                if len(parts) > 1:
                    lemma = parts[1].strip()
                    if lemma:
                         lemma_found = True

    if current_word and not lemma_found and len(current_word) >= 4:
         words_without_lemma.append(current_word)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for word in words_without_lemma:
            outfile.write(word + '\n')


if __name__ == "__main__":
    input_file = 'lemmatized_not_punct_all.txt'
    output_file = 'not_lemma_with_photo.txt'
    find_words_without_lemma(input_file, output_file)
    print(f"Слова без леммы (не короче 4 символов) записаны в файл '{output_file}'.")

#### **4. Лемматизация нелемматизированного остатка с fuzzywuzzy**

In [ ]:
import re
from fuzzywuzzy import process
from uniparser_urmi import UrmiAnalyzer
from collections import defaultdict

def get_all_urmi_lexemes(lexemes_file):
    """
    Читает лексемы из файла словаря урми и возвращает список лексем.
    """
    lexemes = set() # set для быстрой проверки наличия
    try:
        with open(lexemes_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                if line.strip().startswith("lex: "):
                    lexeme = line.strip().replace('lex: ', '')
                    lexemes.add(lexeme)
    except FileNotFoundError:
        print(f"Ошибка: Файл лексем '{lexemes_file}' не найден.")
        return None # если ошибка
    except Exception as e:
        print(f"Ошибка при чтении файла лексем '{lexemes_file}': {e}")
        return None
    return list(lexemes) y

def extract_consonants(word):
    """
    извлекает последовательность согласных из слова.
    """
    return "".join(re.findall(r"[bcdfghjklmnpqrstvwxyz’вƶşçţqk]", word, re.IGNORECASE))

def compare_consonant_sequences(original_word, lexeme):
    """
    Сравнивает последовательности согласных ОРИГИНАЛЬНОГО слова и лексемы.
    Возвращает коэффициент похожести (0-100).
    """
    word_cons = extract_consonants(original_word)
    lexeme_cons = extract_consonants(lexeme)
    if not word_cons or not lexeme_cons:
        return 0

    total_weight = 0
    weighted_match = 0
    word_idx = 0
    lexeme_idx = 0

    while word_idx < len(word_cons) and lexeme_idx < len(lexeme_cons):
        weight = 1.0
        if word_idx < 2: weight = 3.0
        elif word_idx < 3: weight = 2.0

        total_weight += weight

        if word_cons[word_idx].lower() == lexeme_cons[lexeme_idx].lower(): # сравнение без учета регистра
            weighted_match += weight
            lexeme_idx += 1
        word_idx += 1

    return (weighted_match / total_weight) * 100 if total_weight > 0 else 0


def find_best_lemmas(word, urmi_lexemes, limit=10, threshold=60):
    """
    Находит наиболее вероятные леммы для слова, используя нечеткое сравнение и сравнение последовательности согласных.
    Пробует варианты с заменой q - k
    Возвращает список кортежей (лемма, оценка_согласных).
    """
    if not urmi_lexemes: # Если лексемы не загрузились
        return []

    word_lower = word.lower()
    word_variations = {word} # Начинаем с оригинального слова


    
    #генерируем варианты с заменой q - k
    if 'q' in word_lower:
        variant_k = ''.join(['k' if c == 'q' else 'K' if c == 'Q' else c for c in word])
        word_variations.add(variant_k)
    if 'k' in word_lower:
        variant_q = ''.join(['q' if c == 'k' else 'Q' if c == 'K' else c for c in word])
        word_variations.add(variant_q)


    
    combined_results = {} #cловарь для хранения лучших результатов {лемма: лучшаяоценкасогласных}

    # Ищем (оригинал + q/k замены)
    for variant in word_variations:
        # 1й отбор с помощью fuzzywuzzy для текущего вара ( с увелич лимита)
        preliminary_matches = process.extract(variant, urmi_lexemes, limit=limit * 2)
        if not preliminary_matches:
            continue

        # 2. сравнение согласных ОРИГИНАЛЬНОГО слова и выбор подходящих лемм
        for lexeme, fuzzy_score in preliminary_matches:
            # Сравниваем последовательность согласных ОРИГИНАЛЬНОГО слова с лексемой
            consonant_score = compare_consonant_sequences(word, lexeme)

            # Фильтр по порогу согласных и длине
            if consonant_score >= threshold and len(lexeme) >= len(word) / 3:
                #сохраняем или обновляем результат, если текущая оценка лучше
                combined_results[lexeme] = max(combined_results.get(lexeme, 0), consonant_score)

    sorted_matches = sorted(combined_results.items(), key=lambda item: item[1], reverse=True)

    # Возвращаем топ N результатов после объединения и сортировки
    return sorted_matches[:limit]


def find_missing_lemmas(my_words_file, lexemes_file, output_file, limit=10, threshold=60):
    """
    Находит и предлагает леммы для слов, которые не смог распознать UrmiAnalyzer и записывает результаты в файл
    """
    try:
        analyzer = UrmiAnalyzer(mode='nodiacritics')
    except Exception as e:
        print(f"Критическая ошибка: Не удалось инициализировать UrmiAnalyzer: {e}")
        return

    urmi_lexemes = get_all_urmi_lexemes(lexemes_file)
    if urmi_lexemes is None: # Проверка на лексемы
        print("Не удалось загрузить лексемы.")
        return

    urmi_lexemes_set = set(urmi_lexemes) # Используем set для быстрой проверки наличия

    try:
        with open(my_words_file, 'r', encoding='utf-8') as infile, \
             open(output_file, 'w', encoding='utf-8') as outfile:

            for line in infile:
                word = line.strip()
                if not word: continue # Пропускаем пустые строки


                
                try:
                    analyses = analyzer.analyze_words(word)
                    has_lemma = False
                    found_lemma_str = "" # строка для хранения найденной леммы
                    if analyses:
                        for ana in analyses:
                            if hasattr(ana, 'lemma') and ana.lemma:
                                has_lemma = True
                                found_lemma_str = ana.lemma # запомнили первую найденную лемму
                                break # достаточно одной найденной леммы

                    # Если парсер нашёл лемму, записываем
                    if has_lemma:
                         outfile.write(f"Слово: {word}, Найдена лемма (UrmiAnalyzer): {found_lemma_str}\n")
                         continue # переход к следующему слову

                    # если не нашёл
                    best_lemmas = find_best_lemmas(word, urmi_lexemes, limit, threshold)
                    if best_lemmas:
                        # вывод с оценкой согл
                        lemmas_output = ', '.join([f'{lex} ({score:.1f}%)' for lex, score in best_lemmas])
                        outfile.write(f"Слово: {word}, Предложенные леммы (по согласным): {lemmas_output}\n")
                    else:
                        outfile.write(f"Слово: {word}, Не найдено подходящих лемм\n")

                except Exception as e_word:
                    print(f"Ошибка при обработке слова '{word}': {e_word}")
                    outfile.write(f"Слово: {word}, Ошибка обработки: {e_word}\n")

    except FileNotFoundError:
        print(f"Ошибка: Файл со словами '{my_words_file}' не найден.")
    except Exception as e:
        print(f"Ошибка при работе с файлами: {e}")


if __name__ == "__main__":
    my_words_file = "not_lemma_with_photo.txt"
    lexemes_file = "lexemes.txt"                
    output_file = "prob_lemmatized_not_lemma_photo_all.txt" 

    print("Запуск поиска недостающих лемм...")
    find_missing_lemmas(my_words_file, lexemes_file, output_file, limit=10, threshold=60)
    print(f"Поиск завершен. Результаты сохранены в '{output_file}'.")


#### **5. Расфасовка на отлемматизированные слова и всё ещё ошибки**

In [ ]:
# подчищенный

import re
import os
import traceback

# глобальные строчные константы
VOWELS = "aeiouаеёиоуыэюя"
CONSONANTS = set("bcdfghjklmnpqrstvwxyzбвгджзйклмнпрстфхцчшщ")

# ФУНКЦИИ

# для извлечения гласных из слова
def extract_vowels(word):
    return "".join([c.lower() for c in word if c.lower() in VOWELS])

# для расчета схожести гласных
def vowel_similarity(vowels1, vowels2):
    set1 = set(vowels1)
    set2 = set(vowels2)
    if not vowels1 or not vowels2: return 0.0
    common_unique_vowels = len(set1.intersection(set2))
    min_len = min(len(vowels1), len(vowels2))
    return (common_unique_vowels / min_len) * 100 if min_len > 0 else 0.0

# для удаления дубликатов строк
def remove_duplicate_lines(input_file, output_file):
    unique_lines = set()
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile: unique_lines.add(line.strip())
    except FileNotFoundError:
        print(f"Ошибка: Не найден файл: {input_file}")
        return False
    
    sorted_lines = sorted(list(unique_lines))
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in sorted_lines:
                if line: outfile.write(line + '\n')
        return True
    except Exception as e:
        print(f"Ошибка записи: {output_file}: {e}")
        return False

# ОСНОВНОЙ
# для подсчета несовпадающих символов
def count_mismatched_chars(word, lemma):
    word_chars = set(word)
    return sum(1 for char in lemma if char not in word_chars)

# для выбора оптимальной леммы
def choose_best_lemma(word, lemmas_str):
    lemmas = []
    pattern = re.compile(r"([^\s()]+)\s*\((\d+\.?\d*)%\)")
    
    if not lemmas_str:
        return None

    #парсинг строки с вариантами лемм
    for item in lemmas_str.split(", "):
        match = pattern.search(item.strip())
        if match:
            lemma_text = match.group(1)
            try: score = float(match.group(2))
            except ValueError: continue
            lemmas.append((lemma_text, score))

    if not lemmas: return None

    #Выбор леммы с макс %
    try:
        max_score = max(score for _, score in lemmas)
        best_lemmas = [(lemma, score) for lemma, score in lemmas if score == max_score]
        
        if len(best_lemmas) == 1:
            return best_lemmas[0]
            
        # проверка если с одинаковым %
        return min(best_lemmas, key=lambda item: count_mismatched_chars(word, item[0]))
    except:
        return None

# ПРОЦЕСС

#основная функция обработки файла с леммами
def process_lemmatized_file(input_file, all_file, problem_file):
    processed_count = 0
    all_count = 0
    problem_count = 0

    try:
        with open(input_file, 'r', encoding='utf-8') as infile, \
             open(all_file, 'w', encoding='utf-8') as allout, \
             open(problem_file, 'w', encoding='utf-8') as probout:

            # каждая строка
            for line_num, line in enumerate(infile, 1):
                processed_count += 1
                line = line.strip()
                if not line: continue

                # разные типы строк
                if re.match(r"Слово:.*,\s*Найдена лемма \(UrmiAnalyzer\):", line) or \
                   re.match(r"Слово:.*,\s*Найдена лемма \(точное совпадение\):", line) or \
                   ("Предложенная лемма:" in line and "Предложенные леммы" not in line):
                    allout.write(line + '\n')
                    all_count += 1
                    continue
                elif "Не найдено подходящих лемм" in line:
                    probout.write(line + '\n')
                    problem_count += 1
                    continue

                #Обработка предложенных лемм
                elif "Предложенные леммы (по согласным):" in line:
                    # Парсинг слова и вариантов лемм
                    match_word = re.search(r"Слово:\s*([^\s,]+),", line)
                    if not match_word:
                        probout.write(line + ' (Ошибка парсинга слова)\n')
                        problem_count += 1
                        continue
                    word = match_word.group(1)

                    # Доп обработка
                    match_lemmas = re.search(r"Предложенные леммы \(по согласным\):\s*(.+)", line)
                    if not match_lemmas:
                         probout.write(line + ' (Ошибка парсинга лемм)\n')
                         problem_count += 1
                         continue

                    # выбор финальной леммы
                    lemmas_str = match_lemmas.group(1)
                    final_lemma_tuple = None

                    # Спец обработка для слов с определенными ПРЕФИКСАМИ
                    if word and len(word) > 1 and word[0].lower() in 'dlbв' and word[1].lower() in CONSONANTS:
                        modified_word = word[1:]
                        alternative_lemma_tuple = choose_best_lemma(modified_word, lemmas_str)
                        final_lemma_tuple = alternative_lemma_tuple if alternative_lemma_tuple else choose_best_lemma(word, lemmas_str)
                    else:
                        final_lemma_tuple = choose_best_lemma(word, lemmas_str)

                    #Проверка выбранной леммы  
                    if final_lemma_tuple:
                       best_lemma, score = final_lemma_tuple

                       #Проверка гласных для больших %
                       if score >= 80:
                           word_vowels = extract_vowels(word)
                           lemma_vowels = extract_vowels(best_lemma)
                           sim = vowel_similarity(word_vowels, lemma_vowels)
                           vowel_count_diff = len(word_vowels) - len(lemma_vowels)

                           # фильтрует по гласным
                           if sim >= 25 and vowel_count_diff <= 3:
                               allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                               all_count += 1
                           else:
                               reason = f"совпадение гласных {sim:.1f}%"
                               if vowel_count_diff > 3: reason += f", разница гласных {vowel_count_diff}"
                               probout.write(f"Слово: {word}, Лемма: {best_lemma} ({score:.1f}%), не совпало по гласным ({reason})\n")
                               problem_count += 1

                       # Обработывает низкокачественные леммы
                       elif score < 63 and len(word) > 6:
                           probout.write(f"Слово: {word}, Низкая уверенность: {best_lemma} ({score:.1f}%)\n")
                           problem_count += 1
                       else:
                           allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")
                           all_count += 1
                    else:
                        probout.write(f"Слово: {word}, Ошибка выбора леммы из: {lemmas_str}\n")
                        problem_count += 1

                else:
                    probout.write(line + ' (Неопознанный формат)\n')
                    problem_count += 1

        print(f"\nОбработано строк: {processed_count}")
        print(f"Успешные леммы: {all_count}")
        print(f"Проблемные случаи: {problem_count}")

    except FileNotFoundError:
        print(f"Ошибка: Файл не найден: {input_file}")
    except Exception as e:
        print(f"Критическая ошибка: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    input_raw_file = "prob_lemmatized_not_lemma_photo_all.txt"
    unique_file = "prob_lemmatized_not_lemma_photo_unique.txt"
    all_file = "prob_lemmatized_photo_all_final.txt"
    problem_file = "prob_lemmatized_photo_problem.txt"

    if not os.path.exists(input_raw_file):
      print(f"Ошибка: Файл не найден: {input_raw_file}")
      exit(1)

    print(f"Удаление дубликатов...")
    if remove_duplicate_lines(input_raw_file, unique_file):
        print(f"Обработка файла...")
        process_lemmatized_file(unique_file, all_file, problem_file)
        print(f"\nРезультаты:")
        print(f"Успешные леммы: {all_file}")
        print(f"Проблемные случаи: {problem_file}")
    else:
        print("Ошибка обработки")


#### **6. Объединение проблемных слов**

In [ ]:
import re
from collections import defaultdict, Counter
from difflib import SequenceMatcher
import os 

input_problem_file = 'prob_lemmatized_photo_problem.txt'
output_file = 'photo_problems.txt' 

# для вычисления похожести строк
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio() * 100

# Читка проблем и их лемм из файла
def read_problem_words(input_file):
    problem_words = []
    pattern_word = re.compile(r"Слово: ([^,]+),")
    pattern_lemma = re.compile(r"Лемма: ([^\(]+) \((\d+\.?\d*)%\)")
    pattern_line = re.compile(r"Слово: ([^,]+), Лемма: ([^\(]+) \((\d+\.?\d*)%\),?")

    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                #поиск слова и леммы
                m_word = pattern_word.search(line)
                if m_word:
                    word = m_word.group(1)
                    lemmas = []
                    m_lemma = pattern_lemma.findall(line)
                    if m_lemma:
                        for lm in m_lemma:
                            lemmas.append((lm[0].strip(), float(lm[1])))
                    else:
                        # Если леммы нет, поискать в след строке
                        m_line = pattern_line.search(line)
                        if m_line:
                            word = m_line.group(1)
                            lemmas = [(m_line.group(2).strip(), float(m_line.group(3)))]
                        else:
                            lemmas = []
                    problem_words.append({"word": word, "lemmas": lemmas, "line": line})
    except FileNotFoundError:
        print(f"Ошибка: Файл '{input_file}' не найден.")
        return None # если ошибка
    except Exception as e:
        print(f"Ошибка при чтении файла '{input_file}': {e}")
        return None

    return problem_words


#Объединение схожих слов!
def cluster_similar_words(problem_words, similarity_threshold=70):
    clusters = []
    used_indices = set()

    for i, pw1 in enumerate(problem_words):
        if i in used_indices:
            continue
        cluster = [pw1]
        used_indices.add(i)
        for j, pw2 in enumerate(problem_words[i+1:], start=i+1):
            if j in used_indices:
                continue
            sim = similarity(pw1['word'], pw2['word'])
            if sim >= similarity_threshold:
                cluster.append(pw2)
                used_indices.add(j)
        clusters.append(cluster)
    return clusters

# f чтобы выбрать наиболее частотную или по первой по алфавиту форму
def choose_representative(cluster):
    words = [item['word'] for item in cluster]
    freq = Counter(words)
    max_freq = max(freq.values())
    candidates = [w for w, c in freq.items() if c == max_freq]
    return candidates[0]

# Объединение лемм из кластера
def merge_lemmas(cluster):
    lemma_scores = defaultdict(float)
    for item in cluster:
        for lemma, score in item['lemmas']:
            # сохранять макс оценку для леммы
            lemma_scores[lemma] = max(lemma_scores[lemma], score)
    # Сортирует леммы по УБЫВАНИЮ оценки (т.е сначала лучшие)
    sorted_lemmas = sorted(lemma_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_lemmas

# ДЛЯ ИТОГ записи
def format_results(clusters):
    merged_results = []
    for cluster in clusters:
        rep_word = choose_representative(cluster)
        merged_lemmas = merge_lemmas(cluster)
        lemmas_str = ", ".join([f"{lemma} ({score:.1f}%)" for lemma, score in merged_lemmas])
        merged_results.append(f"Слово: {rep_word}, Предложенные леммы: {lemmas_str}")
    return merged_results


# ОСНОВ ЧАСТЬ
if __name__ == "__main__":
    if not os.path.exists(input_problem_file):
        print(f"Ошибка: Файл '{input_problem_file}' не найден.")
        exit(1)

    print(f"Чтение данных из '{input_problem_file}'...")
    problem_words = read_problem_words(input_problem_file)

    if problem_words is None: # на ошибку
        print("Чтение файла прервано из-за ошибки.")
        exit(1)

    # кластеризуем схожие слова
    print("Кластеризация схожих слов...")
    clusters = cluster_similar_words(problem_words)

    # формируем резы
    print("Формирование результатов...")
    formatted_results = format_results(clusters)

    # запись в файл
    print(f"Запись результатов в '{output_file}'...")
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            for line in formatted_results:
                outfile.write(line + '\n')

        print(f"Объединенные результаты сохранены в файл '{output_file}'.")
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_file}': {e}")

#### **7. Восстановление предложений**

(с удалением имен. сущ)

In [ ]:
# почищенный от комментов
import re
import os
from difflib import SequenceMatcher
import traceback
import time

# Именов сущ 
NAMED_ENTITIES_FILE = 'named_entities.txt'
named_entities = set()

try:
    with open(NAMED_ENTITIES_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            entity = line.strip()
            if entity:
                named_entities.add(entity.lower())  # Нормализация для сравнения
        print(f"Загружено {len(named_entities)} уникальных сущностей.")
except Exception as e:
    print(f"Ошибка загрузки сущностей: {e}")


input_main_file = 'lemmatized_not_punct_all.txt'
prob_final_file = 'prob_lemmatized_photo_all_final.txt' 
problems_clustered_file = 'photo_problems.txt'
output_final_sentences_file = 'output_lemmas_sentences_final_new.txt'


def similarity(a, b):
    """вычисление процентной схожести строк"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100 if a and b else 0

def load_prob_final_lemmas(filename):
    """ парсинг файла с предобработанными леммами"""
    lemma_map = {}
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(r"Слово:\s*([^\s,]+)\s*,\s*Предложенная лемма:\s*([^\s\(]+)", line)
                if match:
                    lemma_map[match.group(1).lower()] = match.group(2).strip()
    except Exception as e:
        print(f"Ошибка чтения лемм: {e}")
    return lemma_map

def load_clustered_problem_words(filename):
    """ извлечение проблемных слов для fuzzywuzzy-поиска"""
    word_list = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(r"Слово:\s*([^\s,]+)\s*,", line)
                if match:
                    word_list.append(match.group(1))
    except Exception as e:
        print(f"Ошибка чтения проблемных слов: {e}")
    return word_list

def find_similar_word(target_word, word_list, threshold=70):
    """ поиск приблизительного совпадения с приоритетом строчных вариантов"""
    best_match = None
    for word in word_list:
        if len(word) > 3 and similarity(target_word, word) >= threshold:
            # Предпочтение к нижнему регистру
            if not best_match or word[0].islower():
                best_match = word
                if word[0].islower():
                    break  # Лучший вар найден
    return best_match

# Процесс
def apply_fallback_logic(word, token, prob_lemmas, problem_words, is_first_word):
    """иерархический выбор оптимальной леммы"""
    if not word:
        return token or ""  # защита от пустых значений

    #  1: Именованные сущности
    if word.lower() in named_entities:
        # Удаление сущностей с заглавной буквы
        return None if word[0].isupper() else word
    
    #  2: Предобработанные леммы
    lemma = prob_lemmas.get(word.lower())
    if lemma:
        return word if word[0].isupper() else lemma
    
    # 3: Нечеткий поиск в проблемных словах
    similar = find_similar_word(word.lower() if is_first_word else word, problem_words)
    if similar:
        return similar
    
    # 4-5: Токен без префикса или исходное слово
    return token or word

# ОСНОВА
print(" Инициализация обработки ")
start_time = time.time()

prob_final_lemmas = load_prob_final_lemmas(prob_final_file)
clustered_problem_words = load_clustered_problem_words(problems_clustered_file)

results = []
current_context = {
    "text": None,
    "lemmas": [],
    "current_word": None,
    "current_token": None,
    "is_first_word": False
}

try:
    with open(input_main_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if not line:
                continue

            # Обработка предложений
            if line.startswith("Предложение:"):
                if current_context["text"] and current_context["lemmas"]:
                    results.append({
                        "sentence": current_context["text"], 
                        "lemmas": current_context["lemmas"]
                    })
                current_context = {
                    "text": line.split(": ", 1)[1],
                    "lemmas": [],
                    "current_word": None,
                    "current_token": None,
                    "is_first_word": True
                }

            # Обработка слов
            elif line.startswith("Слово:"):
                if current_context["current_word"]:
                    lemma = apply_fallback_logic(
                        current_context["current_word"],
                        current_context["current_token"],
                        prob_final_lemmas,
                        clustered_problem_words,
                        current_context["is_first_word"]
                    )
                    if lemma is not None:
                        current_context["lemmas"].append(lemma)
                current_context["current_word"] = line.split(": ", 1)[1]
                current_context["current_token"] = None

            # Обработка токенов
            elif line.startswith("Токен:") and current_context["current_word"]:
                current_context["current_token"] = line.split(": ", 1)[1] or current_context["current_word"]

            # Финализация леммы
            elif line.startswith("Лемма:") and current_context["current_word"]:
                lemma = line.split(": ", 1)[1].split('/', 1)[0].strip()
                final_lemma = lemma if len(lemma) >= 4 else apply_fallback_logic(
                    current_context["current_word"],
                    current_context["current_token"],
                    prob_final_lemmas,
                    clustered_problem_words,
                    current_context["is_first_word"]
                )
                if final_lemma is not None:
                    current_context["lemmas"].append(final_lemma)
                current_context["current_word"] = None

    # Финализация последнего предложения
    if current_context["text"] and current_context["lemmas"]:
        results.append({
            "sentence": current_context["text"], 
            "lemmas": current_context["lemmas"]
        })

except Exception as e:
    print(f"Критическая ошибка: {str(e)}")
    traceback.print_exc()
    exit(1)

# Резы
try:
    with open(output_final_sentences_file, 'w', encoding='utf-8') as outfile:
        for item in results:
            outfile.write(' '.join(item['lemmas']) + '\n')
    print(f"Успешно сохранено {len(results)} предложений.")
except Exception as e:
    print(f"Ошибка сохранения: {str(e)}")
    traceback.print_exc()

print(f"Общее время выполнения: {time.time()-start_time:.2f} сек.")

#### **8. Конечная очистка и подсчёт слов**

In [ ]:
import re
import os
import string
import traceback 

def filter_lemmas_and_sentences(input_file, output_file):
    filtered_lines = []
    if not os.path.exists(input_file):
        print(f"Ошибка: Входной файл '{input_file}' не найден.")
        return


    VOWELS_LOWER = set('aeiou')
    LATIN_CONSONANTS_LOWER = set(string.ascii_lowercase) - VOWELS_LOWER

    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            for line_num, line in enumerate(infile, 1): 
                words = line.strip().split()
                processed_words = []

                for word in words:
                    current_word = word.split('/', 1)[0].strip() if '/' in word else word

                    if not current_word: continue 
                    current_word_lower = current_word.lower()

                    word_to_check = current_word_lower 
                    prefix_removed = False 

                    if len(current_word_lower) >= 3 and current_word_lower.startswith(('d', 'l')):
                        char1 = current_word_lower[1]
                        char2 = current_word_lower[2]
                        if char1 in LATIN_CONSONANTS_LOWER and char2 in LATIN_CONSONANTS_LOWER:
                            current_word_lower = current_word_lower[1:] # префикс
                            prefix_removed = True

                    if len(current_word_lower) >= 3:
                        processed_words.append(current_word_lower)


                # количество слов в предложении 
                if len(processed_words) > 2:
                    filtered_lines.append(" ".join(processed_words) + "\n")

    except Exception as e:
        print(f"Ошибка при чтении файла '{input_file}' (строка ~{line_num}): {e}")
        traceback.print_exc() # Добавим вывод стека ошибок
        return

    # рез в файле
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.writelines(filtered_lines)
    except Exception as e:
        print(f"Ошибка при записи в файл '{output_file}': {e}")
        traceback.print_exc()


if __name__ == "__main__":
    input_file = 'output_lemmas_sentences_final_new.txt'
    output_file = 'photo_sentences_filtered_new.txt'

    filter_lemmas_and_sentences(input_file, output_file)

    if os.path.exists(output_file):
         if os.path.getsize(output_file) > 0:
              print(f"Результаты записаны в {output_file}")
         else:
              print(f"Выходной файл {output_file} создан, но он пуст (нет подходящих строк).")
    else:
         print(f"Выходной файл {output_file} не был создан (возможно, из-за ошибки).")

In [ ]:
import re

def count_unique_words(input_file):
    """
    Читает файл и считает количество уникальных слов.
    """
    unique_words = set()
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            words = line.strip().split()
            for word in words:
                unique_words.add(word)
    
    return len(unique_words)


if __name__ == "__main__":
    input_file = 'photo_sentences_filtered_new.txt'
    unique_word_count = count_unique_words(input_file)
    print(f"Количество уникальных слов в файле {input_file}: {unique_word_count}")

__________________________________________________________________________________
слов уникальных - 4719 

предложений - 3445
_____________________________________________________________________

#### **9. Объединение художественных и газетных текстов**

In [ ]:
# Объединение предложений из двух файлов в один

input_file1 = "photo_sentences_filtered_new.txt"
input_file2 = "output_lemmas_sentences_filtered.txt"
output_file = "general_sentences_filtered.txt"

def merge_sentence_files(file1, file2, outfile):
    sentences = []
    for fname in [file1, file2]:
        try:
            with open(fname, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line:
                        sentences.append(line)
        except Exception as e:
            print(f"Ошибка при чтении файла {fname}: {e}")
    try:
        with open(outfile, "w", encoding="utf-8") as out:
            for sent in sentences:
                out.write(sent + "\n")
        print(f"Объединено {len(sentences)} предложений. Результат записан в {outfile}")
    except Exception as e:
        print(f"Ошибка при записи в файл {outfile}: {e}")

if __name__ == "__main__":
    merge_sentence_files(input_file1, input_file2, output_file)

### **Модель, обучение**

In [ ]:
import io

def load_corpus(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    documents = []
    for line in fin:
        documents.append(line.split())
    return documents

In [ ]:
# Эта функция сохраняет словарь (где ключи - слова, а значения - их векторные представления) в файл.
# Это необходимо, чтобы сохранить результаты обучения модели.
def save_dictionary(fname, dictionary, args):
    length, dimension = args # длина словаря и размерность векторов
    fin = io.open(fname, 'w', encoding='utf-8')
    fin.write('%d %d\n' % (length, dimension))
    for word in dictionary:
        fin.write('%s %s\n' % (word, ' '.join(map(str, dictionary[word]))))

# Эта функция читает словарь из файла, созданного с помощью функции save_dictionary.
# Это позволяет загрузить ранее обученную модель.
def load_dictionary(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    length, dimension = map(int, fin.readline().split())
    dictionary = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        dictionary[tokens[0]] = map(float, tokens[1:])
    return dictionary

In [ ]:
documents = load_corpus('general_sentences_filtered.txt')
documents

In [ ]:
len(documents)

In [ ]:
dimension = 80

In [ ]:
%%time
from gensim.models import Word2Vec

#dimension = 8 # размерность векторов слов (эмбеддингов).
#model = Word2Vec(sentences=documents, vector_size=dimension, min_count=1)    было 50 вектор-сайз

model2 = Word2Vec(
    sentences=documents,
    vector_size=dimension,       # размерность
    window=3,            # размер окна
    min_count=1,         # минимальная частота слова
    sg=0,              # CBOW
    negative=15,          # negative sampling
    alpha=0.025,         # learning rate
    min_alpha=0.0001,     # Минимал learning rate
    epochs=30
)

In [ ]:
# создает словарь dictionary, где ключами являются слова, а значениями - их векторные представления.
dictionary = {key : model2.wv[key] for key in model2.wv.key_to_index}

In [ ]:
len(dictionary)

In [ ]:
model2.wv.most_similar('çətun', topn = 100) 

In [ ]:
save_dictionary('urmi_dictionary_general.txt', dictionary, (len(dictionary), dimension))

In [ ]:
loaded_dictionary = load_dictionary('urmi_dictionary_general.txt')
len(dictionary) == len(loaded_dictionary)

In [ ]:
import os
import shutil
import traceback 

def remove_first_line(filepath):
    """
    Удаляет первую строку из указанного файла, тк там прописана размерность векторов
    Создает временный файл, копирует все строки, кроме первой, а затем заменяет исходный файл временным.
    """
    print(f"Попытка удалить первую строку из файла: {filepath}")

    if not os.path.exists(filepath):
        print(f"ОШИБКА: Файл '{filepath}' не найден.")
        return False

    # Создание имени для временного файла  .tmp 
    temp_filepath = filepath + ".tmp"

    try:
        with open(filepath, 'r', encoding='utf-8') as infile, \
             open(temp_filepath, 'w', encoding='utf-8') as outfile:
 
            first_line = infile.readline()

            if not first_line and os.path.getsize(filepath) == 0:
                 print(f"Предупреждение: Файл '{filepath}' пуст. Ничего не удалено.")
                 return True

            shutil.copyfileobj(infile, outfile)

        # замена исходного файла временным
        # проверка на кучу ошибок
        try:
            os.replace(temp_filepath, filepath)
            print(f"Первая строка успешно удалена из файла '{filepath}'.")
            return True
        except OSError as e:
            print(f"ОШИБКА при замене файла '{filepath}' временным файлом '{temp_filepath}': {e}")
            if os.path.exists(temp_filepath):
                try:
                    os.remove(temp_filepath)
                except OSError:
                    print(f"Не удалось удалить временный файл '{temp_filepath}' после ошибки замены.")
            return False

    except FileNotFoundError:
        print(f"ОШИБКА: Файл '{filepath}' не найден во время обработки.")
        return False
    except Exception as e:
        print(f"ОШИБКА при обработке файла '{filepath}': {e}")
        traceback.print_exc()
        if os.path.exists(temp_filepath):
            try:
                os.remove(temp_filepath)
                print(f"Временный файл '{temp_filepath}' удален после ошибки.")
            except OSError:
                 print(f"Не удалось удалить временный файл '{temp_filepath}' после ошибки.")
        return False


if __name__ == "__main__":
    TARGET_FILENAME = "urmi_dictionary_general.txt"

    if remove_first_line(TARGET_FILENAME):
        print("Операция завершена успешно.")
    else:
        print("Операция завершилась с ошибкой.")

#### **Автоматизированный перевод дырок**

In [ ]:
import re
import os
import traceback

def load_translations(lexemes_filepath="lexemes.txt"):
    """
    Загружает переводы (trans_ru) из файла lexemes.txt в словарь.
    Ключи словаря - формы слов из поля 'lex:' в нижнем регистре.
    """
    translations = {}
    
    if not os.path.exists(lexemes_filepath):
        print(f"ОШИБКА: Файл '{lexemes_filepath}' не найден.")
        return translations 
        
    print(f"Загрузка переводов из файла: {lexemes_filepath}")

    # регулярки для извлечения
    lex_pattern = re.compile(r"^\s*lex:\s*(.+)$")
    trans_ru_pattern = re.compile(r"^\s*trans_ru:\s*(.+)$")

    current_lexemes = []
    current_trans_ru = None
    line_num = 0

    
    try:
        with open(lexemes_filepath, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line_stripped = line.strip()

                # начало записи
                if line_stripped.startswith("-lexeme"):
                    # сохранение прошлой записи, если она окей
                    if current_lexemes and current_trans_ru:
                        for lex_form in current_lexemes:
                            # сохранение ключа в нижнем регистре
                            translations[lex_form.lower()] = current_trans_ru
                    #сброс для новой записи
                    current_lexemes = []
                    current_trans_ru = None
                    continue 
                    
                #поиск поля 'lex:'
                lex_match = lex_pattern.match(line_stripped)
                if lex_match and not current_lexemes: # пока не нашли поиск
                    lex_value = lex_match.group(1).strip()
                    variations = [v.strip() for v in lex_value.split('/') if v.strip()]
                    current_lexemes.extend(variations)
                    continue

                #поиск поля 'trans_ru:'
                trans_ru_match = trans_ru_pattern.match(line_stripped)
                if trans_ru_match and not current_trans_ru: 
                    current_trans_ru = trans_ru_match.group(1).strip()
                    continue

            # сохраняем последнее
            if current_lexemes and current_trans_ru:
                for lex_form in current_lexemes:
                    translations[lex_form.lower()] = current_trans_ru

    except Exception as e:
        print(f"ОШИБКА при чтении/парсинге файла '{lexemes_filepath}' (строка ~{line_num}): {e}")
        traceback.print_exc()

    print(f"Загружено переводов для {len(translations)} уникальных форм слов.")
    return translations


def translate_lines_keep_original(input_text, translations_dict):
    """
    Принимает текст и словарь переводов.
    Заменяет слова (разделенные ' | ') на их переводы или оставляет слово как есть.
    Добавляет пустую строку между результатами.
    """
    output_lines = []
    lines = input_text.strip().split('\n') # Разбиваем на строки

    for line in lines:
        # Разбиение строки на слова по '|' и минус пробелы
        original_words = [word.strip() for word in line.split('|') if word.strip()]
        translated_or_original_words = []

        for word in original_words:
            # поиск перевода в словаре
            # Если не найдено, то ОРИГИНАЛЬНОЕ СЛОВО (word)
            processed_word = translations_dict.get(word.lower(), word)
            translated_or_original_words.append(processed_word)

        # собираем строку обратно с разделителем ' | '
        if translated_or_original_words:
             output_lines.append(" | ".join(translated_or_original_words))

    return "\n\n".join(output_lines) # пустая строка между резами



if __name__ == "__main__":
    input_string = """
zukzikkə | məgxuki | вorş | məsruji
ləmsistan | dlitton | prezident | dkomunisti
ruznamь | gavo | qarxa | dvilə
darəçə | vudun | bkişjutə | bǝşəjuvь
vədta | dkule | kəpitalistetə | dqapital
krьsţjana | mşijxəjə | qəddijşi | mədxərtə
səmt | xzuri | pelxə | tupənq | supraxana
məkupi | paşuţь | ţrapa | cjələ
ruznamь | qənunə | ktivjə | bcirij
bxela | pijşɩ | bijovilə | lkis
xləjbə | snədtə | şəxlipə | industrializatsija
cərcərtə | nvəxə | niqo | kəlвə
вrьjra | miko | nьgaranuta | ptiltə
xьjjal | ţmara | pikkir | вləsə
məqdənə | şotapaja | dməxnəqtə | maqţalta
dakrəj | sovxoze | sənjənə | dkolxoz
hoğə | avtomobijl | qujrəti | kombarь | domna
agrotexnikə | bjulpana | texnikə | tusə
sajjarajь | stakan | рənsil | vərəqə
tuç | miljuni | snetə | bəlpi
"""

    # файл с лексемами и переводами
    lexemes_filename = "lexemes.txt"
    #сюда переводы..
    translations_map = load_translations(lexemes_filename)

    if translations_map:
        result_text = translate_lines_keep_original(input_string, translations_map)
        print("\n--- Результат замены ---")
        print(result_text)
    else:
        print("\nЗамена не выполнена из-за ошибки загрузки переводов.")

